# Bernoulli Naive Bayes
***
베르누이 나이브 베이즈를 활용한 스팸 메일 분류
이메일 제목과 레이블(스팸 여부) 데이터를 활용하여 베르누이 나이브 베이즈 분류로 스팸 메일을 분류한다.

In [1]:
import pandas as pd
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 베르누이 나이브 베이즈를 사용하기 위한 라이브러리를 import 한다.
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
# 모델 정확도 평가를 위한 라이브러리를 import 한다.
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

1. 데이터 획득
간단한 스팸 메일 분류를 위해 다음과 같이 이메일 제목과 스팸 메일 레이블이 있는 데이터를 사용한다.
- email title: 이메일 제목
- spam: 스팸 메일 여부

In [3]:
email_list = [
    {'email title': 'free game only today', 'spam': True},
    {'email title': 'cheapest flight deal', 'spam': True},
    {'email title': 'limited time offer only today only today', 'spam': True},
    {'email title': 'today meeting schedule', 'spam': False},
    {'email title': 'your flight schedule attached', 'spam': False},
    {'email title': 'your credit card statement', 'spam': False}
]
df = pd.DataFrame(email_list)
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


2. 학습 데이터 다루기
* scikit-learn의 베르누이 나이브 베이즈 분류기는 <strong>숫자</strong>만 다루기 때문에 True와 False를 1과 0으로 치환한다.
이메일 제목(email title)으로 학습을 진행하고 레이블은 label을 사용해서 스팸 메일 여부를 판단한다.

In [4]:
df['label'] = df['spam'].map({True: 1, False: 0})
# df['label'] = df.get('spam').map({True: 1, False: 0})
# df['label'] = df.spam.map({True: 1, False: 0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [5]:
# 학습에 사용할 데이터와 레이블(클래스, 타겟, 답)로 분리한다.
df_x = df['email title']
df_x

0                        free game only today
1                        cheapest flight deal
2    limited time offer only today only today
3                      today meeting schedule
4               your flight schedule attached
5                  your credit card statement
Name: email title, dtype: object

In [6]:
df_y = df['label']
df_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

★ 베르누이 나이브 베이즈의 입력 데이터는 고정된 크기의 벡터여야만 한다.
scikit-learn의 CountVectorizer() 메서드를 사용하면 데이터에 출현한 모든 단어의 개수 만큼의 크기를 가진 벡터를 만들고 고정된 벡터로 표현한다.

In [7]:
# CountVectorizer() 메서드는 이메일 제목에 출현한 단어를 오름차순으로 정렬하여 단어의 위치로 행렬을 만들어서 리턴한다.
# 특정 단어가 출현할 경우 출현한 단어의 개수를 리턴하고 출현하지 않으면 0을 리턴한다.
# CountVectorizer() 메서드의 옵션으로 binary=True를 지정하면 같은단어가 여러번 출력하더라도 1을 리턴한다.
cv = CountVectorizer(binary=True)
x_train = cv.fit_transform(df_x)
encoded_input = x_train.toarray()  # numpy 타입의 배열 데이터로 변환한다.
print(type(encoded_input))
encoded_input

<class 'numpy.ndarray'>


array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

위의 행렬에서 볼 수 있듯이 이메일 제목에서 총 17개의 단어가 발견되어 각 이메일 제목이 17개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.
베르누이 나이브 베이즈에서 사용하기 위해 이메일 제목에 중복되는 단어가 있더라도 출현한 횟수로 표현되는 것이 아니라 단순이 1로 표현한 것을 알 수 있다.

In [8]:
# inverse_transform() 메서드로 고정된 크기의 벡터에 포함되는 단어를 확인할 수 있다.
cv.inverse_transform(encoded_input)

[array(['free', 'game', 'only', 'today'], dtype='<U9'),
 array(['cheapest', 'deal', 'flight'], dtype='<U9'),
 array(['limited', 'offer', 'only', 'time', 'today'], dtype='<U9'),
 array(['meeting', 'schedule', 'today'], dtype='<U9'),
 array(['attached', 'flight', 'schedule', 'your'], dtype='<U9'),
 array(['card', 'credit', 'statement', 'your'], dtype='<U9')]

In [9]:
# get_feature_names(): 고정된 벡터의 각 열(피쳐)이 어떤 단어를 의미하는지 확인한다.
print(type(cv.get_feature_names_out()))
cv.get_feature_names_out()

<class 'numpy.ndarray'>


array(['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free',
       'game', 'limited', 'meeting', 'offer', 'only', 'schedule',
       'statement', 'time', 'today', 'your'], dtype=object)

# 라플라스 스무딩(Laplace Smoothing)
0이라는 수는 곱셈과 나눗셈을 무력화시키는 값이므로 그 전에 아무리 의미있는 값이 도출된다 하더라도 마지막에 0을 곱해버리면 결과는 0이 나온다. 이런 경우가 상당히 빈번하기 때문에 값이 0이 아닌 최소값(1회 등장)으로 보정을 하게 되는데 이를 라플라스 스무딩이라 한다.

3. 베르누이 나이브 베이즈 모델 학습하기
* scikit-learn의 베르누이 나이브 베이즈 분류기는 기본적으로 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행된다.

In [10]:
bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_train, y_train)

BernoulliNB()

테스트 데이터 다듬고 테스트 실행

In [11]:
# 테스트 데이터
test_email_list = [
    {'email title': 'free flight offer', 'spam': True},
    {'email title': 'hey traveler free flight deal', 'spam': True},
    {'email title': 'limited free game offer', 'spam': True},
    {'email title': 'today flight schedule', 'spam': False},
    {'email title': 'your credit card attached', 'spam': False},
    {'email title': 'free credit card offer only today', 'spam': False}
]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True: 1, False: 0})
test_df

,email title,spam,label
0,free flight offer,True,1
1,hey traveler free flight deal,True,1
2,limited free game offer,True,1
3,today flight schedule,False,0
4,your credit card attached,False,0
5,free credit card offer only today,False,0


In [12]:
test_x = test_df['email title']
test_x

0                    free flight offer
1        hey traveler free flight deal
2              limited free game offer
3                today flight schedule
4            your credit card attached
5    free credit card offer only today
Name: email title, dtype: object

In [13]:
test_y = test_df['label']
test_y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

In [14]:
x_test = cv.transform(test_x)
y_test = test_y.astype(int)

In [15]:
predict = bnb.predict(x_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [16]:
print(f'정확도: {accuracy_score(y_test, predict):.2f}')

정확도: 0.83


In [17]:
comparison = pd.DataFrame({'실제값': y_test, '예측값': predict})
comparison

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,0,0
4,0,0
5,0,1
